In [ ]:

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
import math

# Load training and testing data
ticker = yf.Ticker("MSFT")
data = ticker.history(start='2023-01-01', end='2024-12-31')
dataset = data[['Close']].values

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

training_data_len = int(np.ceil(len(dataset) * 0.9))
train_data = scaled_data[0:training_data_len, :]
test_data = scaled_data[training_data_len - 60:, :]

# Prepare training data by pairing 60-time step sequences with the 61st step
def create_sequences(data, lookback):
    x, y = [], []
    for i in range(lookback, len(data)):
        x.append(data[i-lookback:i, 0])  # 60 previous time steps
        y.append(data[i, 0])             # 61st time step
    return np.array(x), np.array(y)

x_train, y_train = create_sequences(train_data, lookback=60)
x_test, y_test = create_sequences(test_data, lookback=60)

# Reshape inputs to 3D to be LSTM compatible
#     no. samples    =  x_train.shape[0]  -- number of sequences
#     no. time steps =  x_train.shape[1]  -- length of each sequence
#     no. features   =  1                 -- only one feature per time step
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# --------- Build LSTM model ---------
model = Sequential()

# First layer, 128 neurons, outputs sequences to pass to next layer
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))

# Second layer, 64 neurons, only returns final output
model.add(LSTM(units=64, return_sequences=False))

# Intermediate layer
model.add(Dense(units=25))

# Final layer, outputting 1 price prediction
model.add(Dense(units=1))

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(x_train, y_train, batch_size=32, epochs=50)

# --------- Make predictions ---------
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Evaluate predictions
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# --------- Visualize results ---------
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

# Plot
plt.figure(figsize=(16, 6))
plt.title('MSFT Stock Price Prediction (LSTM)')
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.plot(train['Close'], label='Train')
plt.plot(valid[['Close', 'Predictions']], label='Actual')
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()